In [1]:
class Node:
    def __init__(self, value, after=None) -> None:
        self.value = value
        self.after = after


class Stack:
    def __init__(self) -> None:
        self.top = None
        self.size = 0

    def push(self, value):
        self.top = Node(value, self.top)
        self.size += 1

    def pop(self):
        if self.top:
            value = self.top.value
            self.top = self.top.after
            self.size -= 1
            return value
        else:
            raise IndexError("Stack is empty")

    def peek(self):
        if self.top:
            return self.top.value
        else:
            raise IndexError("Stack is empty")

    def clear(self):
        self.top = None
        self.size = 0

    def isEmpty(self):
        return self.__bool__()

    def print(self):
        print(self)

    def __bool__(self):
        return self.size > 0

    def __len__(self):
        return self.size

    def __str__(self):
        return self.__class__.__name__ + ": " + "->".join(map(str, self))

    def __repr__(self):
        return self.__str__()

    def __iter__(self):
        self._iter = self.top
        return self

    def __next__(self):
        if self._iter:
            value = self._iter.value
            self._iter = self._iter.after
            return value
        else:
            raise StopIteration

    def __contains__(self, value):
        for item in self:
            if item == value:
                return True
        return False

In [2]:
def InfixToPostfix(expression: str) -> str:
    operator_precedence = {
        "+": 1,
        "-": 1,
        "*": 2,
        "/": 2,
        "^": 3,
    }

    def precedence(operator):
        return operator_precedence.get(operator, 0)

    operators = Stack()
    output = ""

    for char in expression:
        if char in "0123456789":
            output += char
        elif char in "+-*/^":
            while operators and precedence(operators.peek()) >= precedence(char):
                output += operators.pop()
            operators.push(char)
        elif char == "(":
            operators.push(char)
        elif char == ")":
            while operators.peek() != "(":
                output += operators.pop()
            operators.pop()
        else:
            raise ValueError("Invalid character in expression")
    while operators:
        output += operators.pop()
    return output


def EvaluatePostfix(expression: str) -> int:
    operators = Stack()
    for char in expression:
        if char in "1234567890":
            operators.push(int(char))
        elif char in "+-*/^":
            operand2 = operators.pop()
            operand1 = operators.pop()
            eval_string = f"{operand1}{'**' if char == '^' else char}{operand2}"
            operators.push(eval(eval_string))
        else:
            raise ValueError("Invalid character in expression")

    return operators.pop()  # last item in stack is the result
    ...


def EvaluateEquation(expression: str) -> int:
    return EvaluatePostfix(InfixToPostfix(expression))

In [3]:
infix_expression = "8*(5^4+2)-6^2/(9*3)"
print(f"infix expression: {infix_expression}")
postfix_expression = InfixToPostfix(infix_expression)
print(f"postfix expression: {postfix_expression}")
value = EvaluatePostfix(postfix_expression)
print(f"value: {value}")
print(
    f"result: {infix_expression} = {value} : {value == eval(infix_expression.replace('^', '**'))}"
)

infix expression: 8*(5^4+2)-6^2/(9*3)
postfix expression: 854^2+*62^93*/-
value: 5014.666666666667
result: 8*(5^4+2)-6^2/(9*3) = 5014.666666666667 : True


In [4]:
import numpy as np


def compact_sum(matrix: np.ndarray) -> np.ndarray:
    n = len(matrix)
    x = n // 2
    new_matrix = np.zeros((x, x), dtype=int)
    for i in range(n**2):
        r, c = i // n, i % n
        new_matrix[r // 2][c // 2] += matrix[r][c]
    return new_matrix
    ...


matrix = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 1, 2, 3], [4, 5, 6, 7]])
print(matrix)
new_matrix = compact_sum(matrix)
print(new_matrix)

[[1 2 3 4]
 [5 6 7 8]
 [9 1 2 3]
 [4 5 6 7]]
[[14 22]
 [19 18]]


In [5]:
class LinkedList:
    def __init__(self, arr) -> None:
        self.head = Node(arr[0])
        current = self.head
        for i in range(1, len(arr)):
            new_node = Node(arr[i])
            current.after = new_node
            current = new_node

In [6]:
h1 = LinkedList([1, 2, 3, 4, 5, 6, 7, 9, 8, 10])
h2 = LinkedList([1, 2, 3, 4, 5, 6, 7, 9, 8, 10])


def checkPairSum(h1, h2):
    while h1 and h2:
        if h1.after is None or h2.after is None:
            return False
        if (h1.value + h1.after.value) != (h2.value + h2.after.value):
            return False
        h1 = h1.after.after
        h2 = h2.after.after

    return True


print(checkPairSum(h1.head, h2.head))

True


In [16]:
def confitional_reverse(stack: Stack):
    new_stack = Stack()
    new_stack.push(stack.pop())
    while stack:
        var = stack.pop()
        if var != new_stack.peek():
            new_stack.push(var)

    return new_stack


stack = Stack()
stack.push(1)
stack.push(2)
stack.push(3)
stack.push(3)
stack.push(4)
stack.push(5)
stack.push(5)
stack.push(6)

print(stack)
print(confitional_reverse(stack))

Stack: 6->5->5->4->3->3->2->1
Stack: 1->2->3->4->5->6


In [ ]:
class CallStackItem:
    def __init__(self, func, *args, **kwargs) -> None:
        self.func = func
        self.args = args
        self.kwargs = kwargs


class Try:
    def __init__(self, value=None) -> None:
        self.value = value
        self.error = None
        self.callstack = Stack()

    def Then(self, func, *args, **kwargs):
        try:
            self.value = func(self.value, *args, **kwargs)
        except Exception as e:
            self.error = e

        return self

    def Catch(self, exception=Exception, handler=lambda e: None):
        try:
            return handler(self.value)
        except exception as e:
            return handler(e)

    def Finally(self):
        return self.value

    def __str__(self) -> str:
        return str(self.value)

    def __repr__(self) -> str:
        return self.__str__()